In [ ]:
import pandas as pd 

dtype_dict = {'LAT_MT': str, 'LONG_MT': str}
mt_stop = pd.read_csv('data/agg_mt_gps_with_stop_time_for_surveilance_h30.csv', dtype=dtype_dict, low_memory=False)
display(mt_stop.dtypes)

print("\nJumlah data sebelum drop_duplicates:")
print("Jumlah data (rows):", mt_stop.shape[0])
print("Jumlah kolom:", mt_stop.shape[1])

# Step 1: Drop Duplicate Data
df_mt_stop = mt_stop.drop_duplicates().copy()
print("\nJumlah data setelah drop_duplicates:")
print("Jumlah data (rows):", df_mt_stop.shape[0])
print("Jumlah kolom:", df_mt_stop.shape[1])

# Step 2: Menghapus baris dengan LAMA_BERHENTI_JAM == 0
df_mt_stop['LAMA_BERHENTI_JAM'] = pd.to_numeric(df_mt_stop['LAMA_BERHENTI_JAM'], errors='coerce')
df_mt_stop = df_mt_stop[df_mt_stop['LAMA_BERHENTI_JAM'] != 0]
df_mt_stop = df_mt_stop.dropna(subset=['LAMA_BERHENTI_JAM'])
print("\nJumlah data setelah menghapus LAMA_BERHENTI_JAM == 0:")
print("Jumlah data (rows):", df_mt_stop.shape[0])
print("Jumlah kolom:", df_mt_stop.shape[1])

# Step 3: Mengidentifikasi dan Mengisi Nilai NaN
# a. Mengidentifikasi kolom string (object)
string_columns = df_mt_stop.select_dtypes(include=['object']).columns.tolist()
print("\nKolom bertipe string (object):")
print(string_columns)
df_mt_stop[string_columns] = df_mt_stop[string_columns].fillna('')
print("\nJumlah nilai NaN pada kolom string setelah pengisian:")
print(df_mt_stop[string_columns].isnull().sum())

# b. Mengidentifikasi kolom numerik
numerical_columns = df_mt_stop.select_dtypes(include=['int64', 'float64']).columns.tolist()
print("\nKolom bertipe numerik:")
print(numerical_columns)
df_mt_stop[numerical_columns] = df_mt_stop[numerical_columns].fillna(0)
print("\nJumlah nilai NaN pada kolom numerik setelah pengisian:")
print(df_mt_stop[numerical_columns].isnull().sum())

# Step 4: Memperbaiki nilai LAT_MT dan LONG_MT yang tidak benar
# Pastikan 'LAT_MT' dan 'LONG_MT' bertipe string sebelum menggunakan .str accessor
df_mt_stop['LAT_MT'] = df_mt_stop['LAT_MT'].astype(str)
df_mt_stop['LONG_MT'] = df_mt_stop['LONG_MT'].astype(str)
mask_lat_salah = df_mt_stop['LAT_MT'].str.startswith('.') | df_mt_stop['LAT_MT'].str.startswith('-.')
mask_lon_salah = df_mt_stop['LONG_MT'].str.startswith('.') | df_mt_stop['LONG_MT'].str.startswith('-.')
latitude_salah = df_mt_stop[mask_lat_salah]
longitude_salah = df_mt_stop[mask_lon_salah]
print("\nLatitude yang tidak benar:")
display(latitude_salah)
print("\nLongitude yang tidak benar:")
display(longitude_salah)
df_mt_stop.loc[mask_lat_salah, 'LAT_MT'] = df_mt_stop.loc[mask_lat_salah, 'LAT_MT'].apply(
    lambda x: '0' + x if x.startswith('.') else '-0' + x[1:]
)
df_mt_stop.loc[mask_lon_salah, 'LONG_MT'] = df_mt_stop.loc[mask_lon_salah, 'LONG_MT'].apply(
    lambda x: '0' + x if x.startswith('.') else '-0' + x[1:]
)

df_mt_stop['LAT_MT'] = pd.to_numeric(df_mt_stop['LAT_MT'], errors='coerce')
df_mt_stop['LONG_MT'] = pd.to_numeric(df_mt_stop['LONG_MT'], errors='coerce')
df_mt_stop.to_excel('data/df_mt_stop_cleaned.xlsx', index=False)
print("\nData berhasil diekspor ke 'data/df_mt_stop_cleaned.xlsx'")
print("\nDataFrame setelah perbaikan koordinat:")
display(df_mt_stop)

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd

print("\nSTATISTIK DESKRIPSI")
display(df_mt_stop.describe())

print("\n-MISSING VALUES")
display(df_mt_stop.isnull().sum())

print("\nSTATISTIK DEKSRIPSI BERDASARKAN PRODUK")
display(df_mt_stop.groupby('PRODUK').describe())

plt.figure(figsize=(10, 5))
sns.countplot(x='PRODUK', data=df_mt_stop, palette='Set2')
plt.title('Distribusi Data Berdasarkan PRODUK')
plt.xticks(rotation=45)
plt.show()

In [ ]:
# Step 5: Statistik Deskriptif untuk 'LAMA_BERHENTI_JAM'
print("\nStatistik Deskriptif untuk 'LAMA_BERHENTI_JAM':")
descriptive_stats = df_mt_stop['LAMA_BERHENTI_JAM'].describe()
display(descriptive_stats)

import matplotlib.pyplot as plt
import seaborn as sns

# Membuat histogram dan KDE plot untuk 'LAMA_BERHENTI_JAM'
plt.figure(figsize=(10, 6))
sns.histplot(df_mt_stop['LAMA_BERHENTI_JAM'], bins=30, kde=True, color='skyblue')
plt.title('Distribusi LAMA_BERHENTI_JAM')
plt.xlabel('LAMA_BERHENTI_JAM (Jam)')
plt.ylabel('Frekuensi')
plt.show()


In [ ]:
import geopandas as gpd
from shapely.geometry import Point
import matplotlib.pyplot as plt

df_pertalite = df_mt_stop[df_mt_stop['PRODUK'] == 'PERTALITE']
gdf_mt_pertalite = gpd.GeoDataFrame(df_pertalite, 
                                    geometry=gpd.points_from_xy(df_pertalite['LONG_MT'], df_pertalite['LAT_MT']))

df_solar = df_mt_stop[df_mt_stop['PRODUK'].isin(['BIO SOLAR', 'BIOSOLAR B35'])]
gdf_mt_solar = gpd.GeoDataFrame(df_solar, 
                                geometry=gpd.points_from_xy(df_solar['LONG_MT'], df_solar['LAT_MT']))

gdf_mt_pertalite.set_crs(epsg=4326, inplace=True)
gdf_mt_solar.set_crs(epsg=4326, inplace=True)
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 10))

gdf_mt_pertalite.plot(ax=ax1, marker='o', color='green', markersize=5, alpha=0.7)
ax1.set_title("Mobile Tanker Positions - PERTALITE")

gdf_mt_solar.plot(ax=ax2, marker='o', color='blue', markersize=5, alpha=0.7)
ax2.set_title("Mobile Tanker Positions - BIO SOLAR/B35")

plt.show()

In [ ]:
df_spbu = pd.read_excel("data/iedcc_tbl_spbu_profile.xlsx")

df_spbu_cleaned = df_spbu[(df_spbu['LATITUDE'].between(-90, 90)) & (df_spbu['LONGITUDE'].between(-180, 180))]

gdf_spbu = gpd.GeoDataFrame(df_spbu_cleaned, 
                            geometry=gpd.points_from_xy(df_spbu_cleaned['LONGITUDE'], df_spbu_cleaned['LATITUDE']))

gdf_spbu.set_crs(epsg=4326, inplace=True)
fig, ax = plt.subplots(figsize=(10, 10))
gdf_spbu.plot(ax=ax, marker='x', color='red', markersize=5, alpha=0.7)
plt.title("SPBU Positions")
plt.show()

import geopandas as gpd
from shapely.geometry import Point
import matplotlib.pyplot as plt

gdf_spbu_projected = gdf_spbu.to_crs(epsg=3857)
gdf_spbu_projected['geometry'] = gdf_spbu_projected['geometry'].apply(lambda geom: geom.buffer(500))
gdf_spbu_buffer = gdf_spbu_projected.to_crs(epsg=4326)
gdf_spbu_buffer.columns = [col if len(col) <= 10 else col[:10] for col in gdf_spbu_buffer.columns]
gdf_spbu_buffer.to_file("data/spbu_buffer_500m.shp")

fig, ax = plt.subplots(figsize=(10, 10))
gdf_spbu_buffer.plot(ax=ax, color='red', alpha=0.5)
plt.title("SPBU 500m Buffer")
plt.show()

In [ ]:
df_tbbm = pd.read_excel("data/iedcc_master_data_asset.xlsx")
df_tbbm_cleaned = df_tbbm[(df_tbbm['JENIS_ASSET'] == 'TBBM') &
                          (df_tbbm['LATITUDE'].between(-90, 90)) &
                          (df_tbbm['LONGITUDE'].between(-180, 180))]

gdf_tbbm = gpd.GeoDataFrame(df_tbbm_cleaned, 
                            geometry=gpd.points_from_xy(df_tbbm_cleaned['LONGITUDE'], df_tbbm_cleaned['LATITUDE']))

gdf_tbbm.set_crs(epsg=4326, inplace=True)

fig, ax = plt.subplots(figsize=(10, 10))
gdf_tbbm.plot(ax=ax, marker='s', color='green', markersize=5, alpha=0.7)
plt.title("TBBM Positions")
plt.show()

import geopandas as gpd
from shapely.geometry import Point
import matplotlib.pyplot as plt

gdf_tbbm_projected = gdf_tbbm.to_crs(epsg=3857)

gdf_tbbm_projected['geometry'] = gdf_tbbm_projected['geometry'].apply(lambda geom: geom.buffer(1000))
gdf_tbbm_buffer = gdf_tbbm_projected.to_crs(epsg=4326)
gdf_tbbm_buffer.columns = [col if len(col) <= 10 else col[:10] for col in gdf_tbbm_buffer.columns]
gdf_tbbm_buffer.to_file("data/tbbm_buffer_1km.shp")

fig, ax = plt.subplots(figsize=(10, 10))
gdf_tbbm_buffer.plot(ax=ax, color='red', alpha=0.5)
plt.title("TBBM 1km Buffer")
plt.show()

In [ ]:
mobile_tankers_in_spbu_pertalite = gpd.sjoin(gdf_mt_pertalite, gdf_spbu_buffer, how="inner", predicate="intersects")
mobile_tankers_in_tbbm_pertalite = gpd.sjoin(gdf_mt_pertalite, gdf_tbbm_buffer, how="inner", predicate="intersects")

mobile_tankers_in_spbu_solar = gpd.sjoin(gdf_mt_solar, gdf_spbu_buffer, how="inner", predicate="intersects")
mobile_tankers_in_tbbm_solar = gpd.sjoin(gdf_mt_solar, gdf_tbbm_buffer, how="inner", predicate="intersects")

gdf_mt_filtered_pertalite = gdf_mt_pertalite[
    ~gdf_mt_pertalite.index.isin(mobile_tankers_in_spbu_pertalite.index)
]
gdf_mt_filtered_pertalite = gdf_mt_filtered_pertalite[
    ~gdf_mt_filtered_pertalite.index.isin(mobile_tankers_in_tbbm_pertalite.index)
]

gdf_mt_filtered_solar = gdf_mt_solar[
    ~gdf_mt_solar.index.isin(mobile_tankers_in_spbu_solar.index)
]
gdf_mt_filtered_solar = gdf_mt_filtered_solar[
    ~gdf_mt_filtered_solar.index.isin(mobile_tankers_in_tbbm_solar.index)
]

fig, axes = plt.subplots(1, 2, figsize=(20, 10))

gdf_mt_filtered_pertalite.plot(ax=axes[0], marker='o', color='blue', markersize=5, alpha=0.7)
axes[0].set_title("Filtered Mobile Tankers (Pertalite)")

gdf_mt_filtered_solar.plot(ax=axes[1], marker='o', color='green', markersize=5, alpha=0.7)
axes[1].set_title("Filtered Mobile Tankers (Solar)")

plt.tight_layout()
plt.show()



In [ ]:
from sklearn.cluster import DBSCAN
import numpy as np

coords_pertalite = np.array(list(gdf_mt_filtered_pertalite.geometry.apply(lambda geom: (geom.x, geom.y))))
db_pertalite = DBSCAN(eps=0.01, min_samples=10, metric='euclidean').fit(coords_pertalite)
gdf_mt_filtered_pertalite['cluster'] = db_pertalite.labels_

coords_solar = np.array(list(gdf_mt_filtered_solar.geometry.apply(lambda geom: (geom.x, geom.y))))
db_solar = DBSCAN(eps=0.01, min_samples=10, metric='euclidean').fit(coords_solar)
gdf_mt_filtered_solar['cluster'] = db_solar.labels_

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 10))

gdf_mt_filtered_pertalite.plot(ax=ax1, column='cluster', cmap='tab20', markersize=5, alpha=0.7)
ax1.set_title("DBSCAN Clusters - PERTALITE")

gdf_mt_filtered_solar.plot(ax=ax2, column='cluster', cmap='tab20', markersize=5, alpha=0.7)
ax2.set_title("DBSCAN Clusters - BIO SOLAR/B35")
plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from shapely.geometry import MultiPoint
import geopandas as gpd
from sklearn.cluster import DBSCAN

coords_pertalite = np.array(list(gdf_mt_pertalite.geometry.apply(lambda geom: (geom.x, geom.y))))
db_pertalite = DBSCAN(eps=0.01, min_samples=10, metric='euclidean').fit(coords_pertalite)
gdf_mt_pertalite['cluster'] = db_pertalite.labels_

coords_solar = np.array(list(gdf_mt_solar.geometry.apply(lambda geom: (geom.x, geom.y))))
db_solar = DBSCAN(eps=0.01, min_samples=10, metric='euclidean').fit(coords_solar)
gdf_mt_solar['cluster'] = db_solar.labels_

mean_centers_pertalite = []
mean_centers_solar = []

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 10))

gdf_mt_pertalite.plot(ax=ax1, column='cluster', cmap='tab20', markersize=5, alpha=0.7)
clusters_pertalite = gdf_mt_pertalite['cluster'].unique()
for cluster in clusters_pertalite:
    if cluster != -1:  # Skip noise points (cluster -1)
        cluster_points = gdf_mt_pertalite[gdf_mt_pertalite['cluster'] == cluster]
        multi_point = MultiPoint(cluster_points.geometry.tolist())
        mean_center = multi_point.centroid
        
        mean_centers_pertalite.append({'cluster': cluster, 'geometry': mean_center})
        
        ax1.scatter(mean_center.x, mean_center.y, color='orange', s=50)
ax1.set_title("DBSCAN Clusters & Mean Center - PERTALITE")

gdf_mt_solar.plot(ax=ax2, column='cluster', cmap='tab20', markersize=5, alpha=0.7)
clusters_solar = gdf_mt_solar['cluster'].unique()
for cluster in clusters_solar:
    if cluster != -1:  # Skip noise points (cluster -1)
        cluster_points = gdf_mt_solar[gdf_mt_solar['cluster'] == cluster]
        multi_point = MultiPoint(cluster_points.geometry.tolist())
        mean_center = multi_point.centroid
        
        mean_centers_solar.append({'cluster': cluster, 'geometry': mean_center})
        
        ax2.scatter(mean_center.x, mean_center.y, color='orange', s=50)
ax2.set_title("DBSCAN Clusters & Mean Center - BIO SOLAR/B35")

plt.show()

gdf_mean_centers_pertalite = gpd.GeoDataFrame(mean_centers_pertalite, crs=gdf_mt_pertalite.crs)
gdf_mean_centers_pertalite.to_csv('data/mean_centers_pertalite.csv', index=False)
gdf_mean_centers_pertalite.to_file('data/mean_centers_pertalite.shp')

gdf_mean_centers_solar = gpd.GeoDataFrame(mean_centers_solar, crs=gdf_mt_solar.crs)
gdf_mean_centers_solar.to_csv('data/mean_centers_solar.csv', index=False)
gdf_mean_centers_solar.to_file('data/mean_centers_solar.shp')


In [ ]:
import geopandas as gpd6
import matplotlib.pyplot as plt
from shapely.geometry import MultiPoint

# Function to calculate mean centers, buffer, and plot
def process_mean_center_and_buffer(gdf_mt_filtered, title, ax):
    clusters = gdf_mt_filtered['cluster'].unique()

    mean_centers = []
    cluster_ids = []

    for cluster in clusters:
        if cluster != -1:  # Skip noise points (cluster -1)
            cluster_points = gdf_mt_filtered[gdf_mt_filtered['cluster'] == cluster]

            multi_point = MultiPoint(cluster_points.geometry.tolist())
            mean_center = multi_point.centroid
            mean_centers.append(mean_center)
            cluster_ids.append(cluster)

    # Create GeoDataFrame for mean centers
    gdf_mean_centers = gpd.GeoDataFrame({'cluster': cluster_ids}, geometry=mean_centers, crs="EPSG:4326")

    # Project to EPSG:3857 (meters) and create buffer
    gdf_mean_centers_projected = gdf_mean_centers.to_crs(epsg=3857)
    gdf_mean_center_buffer = gdf_mean_centers_projected.buffer(50)  # 50-meter buffer
    gdf_mean_center_buffer = gpd.GeoSeries(gdf_mean_center_buffer, crs="EPSG:3857").to_crs(epsg=4326)
    gdf_mean_center_buffer_gdf = gpd.GeoDataFrame(geometry=gdf_mean_center_buffer, crs="EPSG:4326")

    # Plot the data
    gdf_mt_filtered.plot(ax=ax, marker='o', color='blue', markersize=5, alpha=0.7)
    gdf_mean_center_buffer_gdf.plot(ax=ax, color='red', alpha=0.5)
    gdf_mean_centers.plot(ax=ax, marker='*', color='yellow', markersize=100, label="Mean Centers")

    ax.set_title(title)
    ax.legend()

# Subplots for side-by-side plotting of PERTALITE and BIOSOLAR/B35
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 10))

# Process and plot PERTALITE
process_mean_center_and_buffer(gdf_mt_pertalite, "50-Meter Buffer Around Mean Centers - PERTALITE", ax1)

# Process and plot BIOSOLAR/B35
process_mean_center_and_buffer(gdf_mt_solar, "50-Meter Buffer Around Mean Centers - BIO SOLAR/B35", ax2)

# Display the plots
plt.show()

In [ ]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from openpyxl.styles import PatternFill, Border, Side
from openpyxl import Workbook
from openpyxl.utils.dataframe import dataframe_to_rows
from openpyxl.utils import get_column_letter
from openpyxl.styles import Font
from matplotlib.lines import Line2D
from matplotlib.patches import Patch
from datetime import datetime, timedelta

def process_mean_center_and_buffer(gdf_mt_filtered, title, ax):
    clusters = gdf_mt_filtered['cluster'].unique()

    mean_centers = []
    cluster_ids = []

    for cluster in clusters:
        if cluster != -1:  # Skip noise points (cluster -1)
            cluster_points = gdf_mt_filtered[gdf_mt_filtered['cluster'] == cluster]

            multi_point = MultiPoint(cluster_points.geometry.tolist())
            mean_center = multi_point.centroid
            mean_centers.append(mean_center)
            cluster_ids.append(cluster)

    gdf_mean_centers = gpd.GeoDataFrame({'cluster': cluster_ids}, geometry=mean_centers, crs="EPSG:4326")
    gdf_mean_centers_projected = gdf_mean_centers.to_crs(epsg=3857)
    gdf_mean_center_buffer = gdf_mean_centers_projected.buffer(100) 
    gdf_mean_center_buffer = gpd.GeoSeries(gdf_mean_center_buffer, crs="EPSG:3857").to_crs(epsg=4326)
    gdf_mean_center_buffer_gdf = gpd.GeoDataFrame(geometry=gdf_mean_center_buffer, crs="EPSG:4326")
    gdf_mt_filtered.plot(ax=ax, marker='o', color='blue', markersize=5, alpha=0.7)
    gdf_mean_center_buffer_gdf.plot(ax=ax, color='red', alpha=0.5)
    gdf_mean_centers.plot(ax=ax, marker='*', color='yellow', markersize=100, label="Mean Centers")

    ax.set_title(title)
    ax.legend()

    now = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
    sanitized_title = title.replace(' ', '_').replace('/', '_').replace('-', '_')
    mean_center_excel_path = f"data/mean_centers_{sanitized_title}_{now}.xlsx"
    gdf_mean_centers.to_excel(mean_center_excel_path, index=False)
    print(f"Mean centers exported to Excel: {mean_center_excel_path}")

    mean_center_shp_path = f"data/mean_centers_{sanitized_title}_{now}.shp"
    gdf_mean_centers.to_file(mean_center_shp_path)
    print(f"Mean centers exported to Shapefile: {mean_center_shp_path}")

    return gdf_mean_centers, gdf_mean_center_buffer_gdf 


def process_intersection(gdf_mt_filtered, gdf_mean_center_buffer_gdf, gdf_mean_centers, ax, file_name):
    gdf_intersect = gpd.sjoin(gdf_mt_filtered, gdf_mean_center_buffer_gdf, how="inner", predicate="intersects")
    
    gdf_intersect = gdf_intersect.merge(gdf_mean_centers[['cluster', 'geometry']], on='cluster', how='left', suffixes=('', '_mean_center'))

    gdf_mt_filtered.plot(ax=ax, marker='o', color='blue', markersize=5, alpha=0.7)
    gdf_mean_center_buffer_gdf.plot(ax=ax, color='red', alpha=0.5)
    gdf_intersect.plot(ax=ax, marker='x', color='green', markersize=100)

    gdf_intersect['LAT_CLUSTER'] = gdf_intersect['geometry_mean_center'].apply(lambda geom: geom.y)
    gdf_intersect['LONG_CLUSTER'] = gdf_intersect['geometry_mean_center'].apply(lambda geom: geom.x)
    gdf_intersect['CLUSTER_ID'] = gdf_intersect['cluster']
    gdf_intersect['BUFFER_DISTANCE'] = 100

    all_columns = list(gdf_intersect.columns) 
    additional_columns = ['LAT_CLUSTER', 'LONG_CLUSTER', 'CLUSTER_ID', 'BUFFER_DISTANCE']
    export_columns = all_columns + additional_columns 
    export_columns = list(dict.fromkeys(export_columns))

    gdf_intersect = gdf_intersect[export_columns]

    if 'geometry' in gdf_intersect.columns:
        df_intersect = pd.DataFrame(gdf_intersect.drop(columns='geometry'))
    else:
        df_intersect = pd.DataFrame(gdf_intersect)

    now = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
    with pd.ExcelWriter(f"data/{file_name}_{now}.xlsx", engine='openpyxl') as writer:
        df_intersect.to_excel(writer, sheet_name=f"mt_stop_{now}", index=False)
        workbook = writer.book
        worksheet = writer.sheets[f"mt_stop_{now}"]

        for col in worksheet.columns:
            max_length = 0
            column = col[0].column_letter
            for cell in col:
                try:
                    if cell.value is not None and len(str(cell.value)) > max_length:
                        max_length = len(str(cell.value))
                except Exception as e:
                    print(f"Error processing cell {cell}: {e}")
            adjusted_width = (max_length + 2)
            worksheet.column_dimensions[column].width = adjusted_width

        thin_border = Border(left=Side(style='thin'), right=Side(style='thin'),
                             top=Side(style='thin'), bottom=Side(style='thin'))
        for row in worksheet.iter_rows():
            for cell in row:
                cell.border = thin_border
                cell.fill = PatternFill(start_color='FFFFFF', end_color='FFFFFF', fill_type='solid')

        worksheet.auto_filter.ref = worksheet.dimensions

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 10))
gdf_mean_centers_pertalite, gdf_mean_center_buffer_gdf_pertalite = process_mean_center_and_buffer(gdf_mt_pertalite, "50-Meter Buffer Around Mean Centers - PERTALITE", ax1)
gdf_mean_centers_solar, gdf_mean_center_buffer_gdf_solar = process_mean_center_and_buffer(gdf_mt_solar, "50-Meter Buffer Around Mean Centers - BIO SOLAR/B35", ax2)

fig, (ax3, ax4) = plt.subplots(1, 2, figsize=(20, 10))
process_intersection(gdf_mt_pertalite, gdf_mean_center_buffer_gdf_pertalite, gdf_mean_centers_pertalite, ax3, "pertalite")
process_intersection(gdf_mt_solar, gdf_mean_center_buffer_gdf_solar, gdf_mean_centers_solar, ax4, "biosolar")
legend_elements = [
    Line2D([0], [0], marker='o', color='w', markerfacecolor='blue', markersize=10, label='Mobile Tankers'),
    Patch(facecolor='red', edgecolor='red', label='100m Buffers'),
    Line2D([0], [0], marker='x', color='w', markerfacecolor='green', markersize=10, label='Intersected Points')
]
ax3.legend(handles=legend_elements)
ax3.set_title("Spatial Intersection Between Mobile Tankers and 50-Meter Buffers - PERTALITE")
ax4.legend(handles=legend_elements)
ax4.set_title("Spatial Intersection Between Mobile Tankers and 50-Meter Buffers - BIO SOLAR/B35")

plt.show()